# 🧠 Psychiatry Vector Arithmetic Laboratory

An interactive laboratory for exploring neural representations in therapeutic contexts using vector arithmetic. This notebook allows you to experiment with positive thought patterns, cognitive transformations, and advanced activation manipulation.

## 🔬 Lab Features:
- Load and experiment with positive thought patterns from JSONL dataset
- Multi-layer activation extraction and injection
- Advanced vector arithmetic operations
- Modular parameter controls for easy experimentation
- Custom interpretation prompts for therapeutic contexts
- Interactive visualization of cognitive pattern transformations

## 🔧 Lab Setup and Initialization

Initialize all required components. Run this once, then experiment freely below.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install torch transformers nnsight tqdm pandas numpy matplotlib seaborn
# !pip install accelerate  # For efficient model loading

# FOR AMD GPU
# import os
# os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"
# os.environ["HIP_VISIBLE_DEVICES"] = "0"
# os.environ["AMD_SERIALIZE_KERNEL"] = "3"
# os.environ["TORCH_USE_HIP_DSA"] = "1"

# IMPORTS
import sys
import os
import warnings
import json
import random
from pathlib import Path

# Add current project directory to path to import nnsight_selfie
sys.path.insert(0, '..')  # Go up one level from examples/ to the project root
warnings.filterwarnings('ignore', category=FutureWarning)

# Import our library
from nnsight_selfie import (
    ModelAgnosticSelfie, 
    InterpretationPrompt, 
    print_device_info, 
    get_optimal_device
)

# Standard imports
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from typing import List, Dict, Any, Tuple, Optional
from collections import defaultdict
import itertools

torch.set_grad_enabled(False)

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("✅ Psychiatry Vector Arithmetic Lab initialized!")
print("🧠 Ready to explore therapeutic neural representations")

✅ Psychiatry Vector Arithmetic Lab initialized!
🧠 Ready to explore therapeutic neural representations


In [2]:
# Show device information and load model
print("=== Device Detection ===")
print_device_info()

optimal_device = get_optimal_device()
print(f"🚀 Optimal device: {optimal_device}")

# Load model (adjust model name as needed)
MODEL_NAME = "google/gemma-3-4b-it"  # Change this to experiment with different models

print(f"\n📥 Loading {MODEL_NAME}...")
print("This may take a few minutes on first run")

try:
    selfie = ModelAgnosticSelfie(
        MODEL_NAME,
        dtype=torch.bfloat16,
        load_in_8bit=False
    )
    
    print(f"✅ Model loaded on: {selfie.device}")
    print(f"📊 Layers: {len(selfie.layer_paths)}")
    print(f"🔤 Vocab: {len(selfie.model.tokenizer):,} tokens")
    
    # Store model info for experiments
    MODEL_INFO = {
        'name': MODEL_NAME,
        'device': selfie.device,
        'total_layers': len(selfie.layer_paths),
        'vocab_size': len(selfie.model.tokenizer)
    }
    
except Exception as e:
    print(f"❌ Failed to load {MODEL_NAME}: {e}")
    print("🔄 Consider using a smaller model or checking memory")
    raise

=== Device Detection ===
=== Device Information ===
Platform: Linux x86_64
Python: 3.12.3
PyTorch: 2.4.1+rocm6.0
Optimal Device: cuda

=== MPS Support ===
MPS Available: False
MPS Built: False

=== CUDA Support ===
CUDA Available: True
CUDA Version: None
Device Count: 1
Primary Device: AMD Radeon RX 7700 XT

🚀 Optimal device: cuda

📥 Loading google/gemma-3-4b-it...
This may take a few minutes on first run
Initializing model on device: cuda
Filtered out vision components for Gemma 3 4B model.
Model loaded successfully with 35 layers detected.
✅ Model loaded on: cuda
📊 Layers: 35
🔤 Vocab: 262,145 tokens


In [ ]:
# Hardcoded positive patterns for psychiatry experiments
print("📂 Loading hardcoded positive patterns for psychiatry experiments")

# Define hardcoded psychiatry-related patterns
positive_patterns = [
    {
        'cognitive_pattern_type': 'Negative self-evaluative loop',
        'cognitive_pattern_name': 'Self-Critical Rumination',
        'pattern_description': 'Breaking cycles of harsh self-judgment and criticism',
        'positive_thought_pattern': 'I notice I\'m being harsh on myself right now. Instead of focusing on what went wrong, I can acknowledge that making mistakes is part of learning and growing. I\'m doing my best with the resources and knowledge I have today.',
        'reference_negative_example': 'I\'m such an idiot for making that mistake. I always mess things up. Why can\'t I ever do anything right? Everyone else seems to have it figured out except me.'
    },
    {
        'cognitive_pattern_type': 'Anxiety catastrophizing',
        'cognitive_pattern_name': 'Anxiety Spiral Interruption', 
        'pattern_description': 'Interrupting catastrophic thinking patterns with grounded reality',
        'positive_thought_pattern': 'I can feel my anxiety starting to spiral into worst-case scenarios. Let me pause and breathe. Most of what I\'m worrying about hasn\'t happened yet and may never happen. I can focus on what I can control right now.',
        'reference_negative_example': 'What if everything goes wrong? What if I fail completely? What if people think I\'m incompetent? This is going to be a disaster and I\'ll never recover from it.'
    },
    {
        'cognitive_pattern_type': 'Depression hopelessness',
        'cognitive_pattern_name': 'Hope Cultivation',
        'pattern_description': 'Cultivating hope and possibility during depressive episodes',
        'positive_thought_pattern': 'Even though things feel really dark right now, I\'ve gotten through difficult times before. This feeling won\'t last forever. I can take one small step today, even if it\'s just getting dressed or going for a short walk.',
        'reference_negative_example': 'Nothing will ever get better. I\'m stuck in this misery forever. There\'s no point in trying because everything I do fails anyway. Why even bother?'
    },
    {
        'cognitive_pattern_type': 'Perfectionism paralysis',
        'cognitive_pattern_name': 'Progress Over Perfection',
        'pattern_description': 'Embracing progress and learning over perfectionist standards',
        'positive_thought_pattern': 'I don\'t need to be perfect to be valuable or to make progress. Done is better than perfect. I can start with a rough draft and improve it over time. Every step forward counts, no matter how small.',
        'reference_negative_example': 'If I can\'t do this perfectly, there\'s no point in doing it at all. Everyone will see my flaws and judge me. I need to wait until I can guarantee a perfect outcome.'
    },
    {
        'cognitive_pattern_type': 'Social anxiety avoidance',
        'cognitive_pattern_name': 'Social Courage Building',
        'pattern_description': 'Building confidence and reducing avoidance in social situations',
        'positive_thought_pattern': 'Social situations feel challenging, but I can handle them. Most people are focused on themselves, not judging me. I have valuable things to contribute to conversations. I can start with small interactions and build my confidence.',
        'reference_negative_example': 'Everyone is looking at me and thinking negative things. I\'ll say something stupid and embarrass myself. It\'s safer to just avoid social situations altogether.'
    },
    {
        'cognitive_pattern_type': 'Trauma hypervigilance',
        'cognitive_pattern_name': 'Safety and Grounding',
        'pattern_description': 'Creating safety and presence after trauma responses',
        'positive_thought_pattern': 'I notice my body is on high alert right now. That\'s my nervous system trying to protect me. I\'m safe in this moment. I can feel my feet on the ground and take some deep breaths to help my system calm down.',
        'reference_negative_example': 'Danger is everywhere. I need to stay constantly alert or something terrible will happen. I can\'t trust anyone or anything. The world is not safe.'
    },
    {
        'cognitive_pattern_type': 'Emotional overwhelm',
        'cognitive_pattern_name': 'Emotional Regulation',
        'pattern_description': 'Managing intense emotions with self-compassion and coping skills',
        'positive_thought_pattern': 'These emotions feel intense right now, but they will pass like clouds in the sky. I can acknowledge what I\'m feeling without being consumed by it. Let me use my breathing techniques and remind myself that feelings are temporary.',
        'reference_negative_example': 'I can\'t handle these emotions. They\'re too overwhelming. I\'m completely out of control and there\'s nothing I can do about it. These feelings will never end.'
    },
    {
        'cognitive_pattern_type': 'Mindfulness grounding',
        'cognitive_pattern_name': 'Present Moment Awareness',
        'pattern_description': 'Cultivating present-moment awareness and mindful attention',
        'positive_thought_pattern': 'Right now, in this moment, I am breathing. I can feel the air moving in and out of my lungs. I can notice the sounds around me, the feeling of my body in this chair. This present moment is all I need to focus on right now.',
        'reference_negative_example': 'My mind is racing with thoughts about the past and future. I can\'t focus on anything. Everything feels chaotic and scattered. I have no control over my thoughts.'
    },
    {
        'cognitive_pattern_type': 'Self-compassion building',
        'cognitive_pattern_name': 'Inner Kindness',
        'pattern_description': 'Developing a kind and understanding relationship with oneself',
        'positive_thought_pattern': 'I\'m going through a difficult time, and it\'s okay to struggle. If my best friend were in this situation, I would offer them kindness and understanding. I can offer myself that same compassion. I\'m human, and humans have challenges.',
        'reference_negative_example': 'I should be stronger than this. Other people don\'t struggle like I do. I\'m weak and pathetic for having these problems. I don\'t deserve kindness or understanding.'
    },
    {
        'cognitive_pattern_type': 'Resilience building',
        'cognitive_pattern_name': 'Strength Recognition',
        'pattern_description': 'Recognizing personal strength and resilience capacity',
        'positive_thought_pattern': 'I\'ve overcome challenges before, which shows I have strength and resilience within me. Even when things are hard, I find ways to cope and continue. I can trust in my ability to handle whatever comes my way.',
        'reference_negative_example': 'I\'m not strong enough for this. I always fall apart when things get difficult. I have no resilience or coping skills. I\'ll never be able to handle life\'s challenges.'
    }
]

print(f"✅ Loaded {len(positive_patterns)} hardcoded positive thought patterns")

# Show pattern categories
pattern_types = defaultdict(int)
pattern_names = defaultdict(int)

for pattern in positive_patterns:
    pattern_types[pattern['cognitive_pattern_type']] += 1
    pattern_names[pattern['cognitive_pattern_name']] += 1

print(f"\n📊 Pattern Distribution:")
print(f"  Types: {len(pattern_types)} unique types")
print(f"  Names: {len(pattern_names)} unique patterns")

# Show all pattern types
print(f"\n🔝 Pattern Types:")
for ptype, count in sorted(pattern_types.items(), key=lambda x: x[1], reverse=True):
    print(f"  - {ptype}: {count} examples")

## 🎭 Therapeutic Interpretation Prompts

Create specialized interpretation prompts for therapeutic and psychological contexts.

In [4]:
# Create therapeutic interpretation prompts
THERAPEUTIC_PROMPTS = {
    "cognitive_pattern": InterpretationPrompt(
        selfie.model.tokenizer,
        ["This neural pattern represents the cognitive process of ", None]
    ),
    
    "emotional_state": InterpretationPrompt(
        selfie.model.tokenizer,
        ["This activation encodes the emotional state of ", None]
    ),
    
    "therapeutic_concept": InterpretationPrompt(
        selfie.model.tokenizer,
        ["In therapeutic terms, this represents ", None]
    ),
    
    "mindfulness": InterpretationPrompt(
        selfie.model.tokenizer,
        ["From a mindfulness perspective, this embodies ", None]
    ),
    
    "resilience": InterpretationPrompt(
        selfie.model.tokenizer,
        ["This neural signature reflects resilience through ", None]
    ),
    
    "self_compassion": InterpretationPrompt(
        selfie.model.tokenizer,
        ["This pattern demonstrates self-compassion by ", None]
    ),
    
    "growth_mindset": InterpretationPrompt(
        selfie.model.tokenizer,
        ["This activation shows growth mindset through ", None]
    ),
    
    "coping_mechanism": InterpretationPrompt(
        selfie.model.tokenizer,
        ["As a coping mechanism, this represents ", None]
    )
}

print("🎭 Therapeutic Interpretation Prompts Created:")
for name, prompt in THERAPEUTIC_PROMPTS.items():
    print(f"  {name.replace('_', ' ').title()}: '{prompt.get_prompt()}'")

print(f"\n✅ {len(THERAPEUTIC_PROMPTS)} therapeutic prompts ready for experimentation")

🎭 Therapeutic Interpretation Prompts Created:
  Cognitive Pattern: 'This neural pattern represents the cognitive process of _ '
  Emotional State: 'This activation encodes the emotional state of _ '
  Therapeutic Concept: 'In therapeutic terms, this represents _ '
  Mindfulness: 'From a mindfulness perspective, this embodies _ '
  Resilience: 'This neural signature reflects resilience through _ '
  Self Compassion: 'This pattern demonstrates self-compassion by _ '
  Growth Mindset: 'This activation shows growth mindset through _ '
  Coping Mechanism: 'As a coping mechanism, this represents _ '

✅ 8 therapeutic prompts ready for experimentation


## 🧪 Experimental Utilities

Modular functions for easy experimentation with different parameters.

In [5]:
class PsychiatryVectorLab:
    """Main laboratory class for psychiatric vector arithmetic experiments."""
    
    def __init__(self, selfie_model, positive_patterns, therapeutic_prompts):
        self.selfie = selfie_model
        self.patterns = positive_patterns
        self.prompts = therapeutic_prompts
        self.activation_cache = {}
        
    def get_random_pattern(self, pattern_type: Optional[str] = None, pattern_name: Optional[str] = None):
        """Get a random positive pattern, optionally filtered by type or name."""
        candidates = self.patterns.copy()
        
        if pattern_type:
            candidates = [p for p in candidates if pattern_type.lower() in p['cognitive_pattern_type'].lower()]
        
        if pattern_name:
            candidates = [p for p in candidates if pattern_name.lower() in p['cognitive_pattern_name'].lower()]
        
        if not candidates:
            print(f"⚠️ No patterns found matching criteria: type='{pattern_type}', name='{pattern_name}'")
            return None
            
        return random.choice(candidates)
    
    def tokenize_and_display(self, text: str, max_tokens: int = 20) -> Tuple[List[int], List[str]]:
        """Tokenize text and display tokens for selection."""
        tokens = self.selfie.model.tokenizer.encode(text)
        token_strings = [self.selfie.model.tokenizer.decode([t]) for t in tokens]
        
        print(f"\n🔤 Tokenization ({len(tokens)} tokens):")
        display_limit = min(len(tokens), max_tokens)
        
        for i in range(display_limit):
            token_str = token_strings[i]
            print(f"  {i:2d}: '{token_str.strip()}'")
        
        if len(tokens) > max_tokens:
            print(f"  ... and {len(tokens) - max_tokens} more tokens")
            
        return tokens, token_strings
    
    def extract_activations(self, 
                          text: str,
                          layers: List[int],
                          token_positions: List[int],
                          cache_key: Optional[str] = None) -> Dict[int, List[torch.Tensor]]:
        """Extract activations from multiple layers and token positions."""
        
        if cache_key and cache_key in self.activation_cache:
            print(f"📋 Using cached activations: {cache_key}")
            return self.activation_cache[cache_key]
        
        print(f"🧮 Extracting activations from {len(layers)} layers, {len(token_positions)} tokens...")
        
        activations = self.selfie.get_activations(
            text,
            layer_indices=layers,
            token_indices=token_positions
        )
        
        if cache_key:
            self.activation_cache[cache_key] = activations
            print(f"💾 Cached activations: {cache_key}")
        
        return activations
    
    def interpret_activation(self,
                           text: str,
                           layer: int,
                           token_position: int,
                           prompt_name: str = "cognitive_pattern",
                           max_tokens: int = 15) -> str:
        """Interpret a single activation using specified therapeutic prompt."""
        
        if prompt_name not in self.prompts:
            available = list(self.prompts.keys())
            print(f"❌ Unknown prompt: {prompt_name}. Available: {available}")
            return "[Error: Unknown prompt]"
        
        try:
            result = self.selfie.interpret(
                original_prompt=text,
                interpretation_prompt=self.prompts[prompt_name],
                tokens_to_interpret=[(layer, token_position)],
                max_new_tokens=max_tokens
            )
            
            return result['interpretation'][0].strip()
            
        except Exception as e:
            print(f"❌ Interpretation failed: {e}")
            return f"[Error: {str(e)}]"
    
    def vector_arithmetic(self,
                         base_activations: Dict[int, List[torch.Tensor]],
                         base_layer: int,
                         base_token: int,
                         operations: List[Tuple[str, Dict, int, int, float]]) -> torch.Tensor:
        """Perform vector arithmetic: base + sum(op_weight * activation).
        
        Args:
            base_activations: Activations dict from base text
            base_layer: Layer index for base vector
            base_token: Token index for base vector
            operations: List of (operation, activations_dict, layer, token, weight)
                       where operation is "+" or "-"
        """
        
        # Get base vector
        if isinstance(base_activations[base_layer], list):
            result = base_activations[base_layer][base_token].clone()
        else:
            result = base_activations[base_layer][:, base_token, :].clone()
        
        print(f"🔢 Starting with base vector from layer {base_layer}, token {base_token}")
        
        # Apply operations
        for op, activations, layer, token, weight in operations:
            if isinstance(activations[layer], list):
                vector = activations[layer][token]
            else:
                vector = activations[layer][:, token, :]
            
            if op == "+":
                result = result + weight * vector
                print(f"  ➕ Added {weight}x vector from layer {layer}, token {token}")
            elif op == "-":
                result = result - weight * vector
                print(f"  ➖ Subtracted {weight}x vector from layer {layer}, token {token}")
            else:
                print(f"  ⚠️ Unknown operation: {op}")
        
        return result
    
    def interpret_vector(self,
                        vector: torch.Tensor,
                        injection_layer: int,
                        prompt_name: str = "cognitive_pattern",
                        max_tokens: int = 20) -> str:
        """Interpret a computed vector by injecting it into the model."""
        
        if prompt_name not in self.prompts:
            return f"[Error: Unknown prompt {prompt_name}]"
        
        try:
            interpretation = self.selfie.interpret_vectors(
                vectors=[vector],
                interpretation_prompt=self.prompts[prompt_name],
                injection_layer=injection_layer,
                max_new_tokens=max_tokens
            )[0]
            
            return interpretation.strip()
            
        except Exception as e:
            return f"[Error: {str(e)}]"

# Initialize the lab
lab = PsychiatryVectorLab(selfie, positive_patterns, THERAPEUTIC_PROMPTS)
print("🧪 Psychiatry Vector Lab initialized and ready for experiments!")

🧪 Psychiatry Vector Lab initialized and ready for experiments!


## 📊 Dataset Explorer

Explore the positive patterns dataset to understand available therapeutic contexts.

In [6]:
# Explore the dataset patterns
print("📊 Dataset Explorer")
print("=" * 50)

if positive_patterns:
    # Show pattern types distribution
    pattern_type_counts = defaultdict(int)
    for pattern in positive_patterns:
        pattern_type_counts[pattern['cognitive_pattern_type']] += 1
    
    print(f"\n🧠 Cognitive Pattern Types ({len(pattern_type_counts)} total):")
    for ptype, count in sorted(pattern_type_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {ptype:.<35} {count:>3} examples")
    
    # Show some example patterns
    print(f"\n📝 Example Patterns:")
    for i, pattern in enumerate(positive_patterns[:3]):
        print(f"\n🔸 Example {i+1}: {pattern['cognitive_pattern_name']}")
        print(f"  Type: {pattern['cognitive_pattern_type']}")
        print(f"  Positive Pattern: {pattern['positive_thought_pattern'][:100]}...")
        if 'reference_negative_example' in pattern:
            print(f"  Negative Example: {pattern['reference_negative_example'][:80]}...")
    
    # Interactive pattern selector
    print(f"\n🎯 Quick Pattern Access:")
    print("Use lab.get_random_pattern() or specify filters:")
    print("  - lab.get_random_pattern(pattern_type='rumination')")
    print("  - lab.get_random_pattern(pattern_name='Self-Critical')")
    
else:
    print("❌ No patterns loaded. Please check the dataset path.")

📊 Dataset Explorer

🧠 Cognitive Pattern Types (13 total):
  Cognitive depletion pattern........  40 examples
  Intrusive suicidal fixation........  40 examples
  Negative self-evaluative loop......  40 examples
  Internal dialectical processing....  40 examples
  Fragmented perceptual reasoning....  40 examples
  Hyper-attuned interoception........  40 examples
  Autobiographical integration.......  40 examples
  Over-elaborative recounting........  40 examples
  Entrapment cognition...............  40 examples
  Existential rumination.............  40 examples
  Learned helplessness loop..........  40 examples
  Instrumental suicidal reasoning....  40 examples
  Cognitive disorganization..........  40 examples

📝 Example Patterns:

🔸 Example 1: Executive Fatigue & Avolition
  Type: Cognitive depletion pattern
  Positive Pattern: I'm recognizing that my energy levels are flagging today, which is totally normal. I've been pushing...
  Negative Example: Ugh, just the thought of checking 

## 🔬 Experiment 1: Single Pattern Analysis

Analyze individual positive thought patterns across different layers and tokens.

In [7]:
# ===== EXPERIMENT PARAMETERS =====
# Modify these to change the experiment

# Pattern selection (set to None for random)
EXPERIMENT_PATTERN_TYPE = None  # e.g., "rumination", "intrusive", "negative self-evaluative"
EXPERIMENT_PATTERN_NAME = None  # e.g., "Self-Critical", "Executive Fatigue"

# Layer analysis parameters  
ANALYSIS_LAYERS = [5, 10, 15, 20, 25, 30]  # Which layers to analyze
MAX_TOKENS_TO_SHOW = 90  # How many tokens to display for selection

# Interpretation parameters
INTERPRETATION_PROMPT = "cognitive_pattern"  # Which therapeutic prompt to use
MAX_INTERPRETATION_TOKENS = 20

# ===== EXPERIMENT EXECUTION =====

print("🔬 EXPERIMENT 1: Single Pattern Analysis")
print("=" * 60)

# Get pattern
pattern = lab.get_random_pattern(
    pattern_type=EXPERIMENT_PATTERN_TYPE,
    pattern_name=EXPERIMENT_PATTERN_NAME
)

if not pattern:
    print("❌ No pattern found. Please adjust your filters.")
else:
    print(f"🎯 Selected Pattern: {pattern['cognitive_pattern_name']}")
    print(f"📝 Type: {pattern['cognitive_pattern_type']}")
    print(f"📄 Description: {pattern['pattern_description']}")
    
    # Use the positive thought pattern
    text = pattern['positive_thought_pattern']
    print(f"\n💭 Analyzing: '{text[:100]}...'")
    
    # Tokenize and show options
    tokens, token_strings = lab.tokenize_and_display(text, MAX_TOKENS_TO_SHOW)
    
    # Select interesting tokens (modify these based on tokenization output above)
    INTERESTING_TOKENS = [2, 4, 6, -3, -2]  # Adjust based on your tokenization
    
    # Convert negative indices
    token_positions = []
    for pos in INTERESTING_TOKENS:
        if pos < 0:
            token_positions.append(len(tokens) + pos)
        else:
            token_positions.append(pos)
    
    # Filter valid positions
    token_positions = [p for p in token_positions if 0 <= p < len(tokens)]
    
    print(f"\n🎯 Analyzing tokens at positions: {token_positions}")
    for pos in token_positions:
        if pos < len(token_strings):
            print(f"  {pos:2d}: '{token_strings[pos].strip()}'")
    
    # Extract activations
    activations = lab.extract_activations(
        text=text,
        layers=ANALYSIS_LAYERS,
        token_positions=token_positions,
        cache_key=f"single_pattern_{pattern['cognitive_pattern_name'][:20]}"
    )
    
    print(f"\n🧠 Interpretations using '{INTERPRETATION_PROMPT}' prompt:")
    print("=" * 80)
    
    # Interpret key combinations
    for layer in [10, 15, 25]:  # Focus on a few layers
        for token_idx, token_pos in enumerate(token_positions[:3]):  # First 3 tokens
            if token_pos < len(token_strings):
                token_text = token_strings[token_pos].strip()
                interpretation = lab.interpret_activation(
                    text=text,
                    layer=layer,
                    token_position=token_pos,
                    prompt_name=INTERPRETATION_PROMPT,
                    max_tokens=MAX_INTERPRETATION_TOKENS
                )
                
                print(f"Layer {layer:2d}, Token {token_pos:2d} ('{token_text}'):")
                print(f"  🤖 {interpretation}")
                print()
    
    print("✅ Single pattern analysis complete!")

🔬 EXPERIMENT 1: Single Pattern Analysis
🎯 Selected Pattern: Persistent Suicidal Ideation Focus
📝 Type: Intrusive suicidal fixation
📄 Description: Thoughts repeatedly return to death as the central theme, crowding out other cognition.

💭 Analyzing: 'I've noticed that my mind tends to wander back to the possibility of death, but I'm taking it as a s...'

🔤 Tokenization (125 tokens):
   0: '<bos>'
   1: 'I'
   2: '''
   3: 've'
   4: 'noticed'
   5: 'that'
   6: 'my'
   7: 'mind'
   8: 'tends'
   9: 'to'
  10: 'wander'
  11: 'back'
  12: 'to'
  13: 'the'
  14: 'possibility'
  15: 'of'
  16: 'death'
  17: ','
  18: 'but'
  19: 'I'
  20: '''
  21: 'm'
  22: 'taking'
  23: 'it'
  24: 'as'
  25: 'a'
  26: 'sign'
  27: 'that'
  28: 'I'
  29: '''
  30: 'm'
  31: 'still'
  32: 'grappling'
  33: 'with'
  34: 'some'
  35: 'deep'
  36: '-'
  37: 'seated'
  38: 'concerns'
  39: 'and'
  40: 'fears'
  41: '.'
  42: 'Rather'
  43: 'than'
  44: 'getting'
  45: 'stuck'
  46: 'on'
  47: 'them'
  48: ','
 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You have set `compile_config`, but we are unable to meet the criteria for compilation. Compilation will be skipped.


💾 Cached activations: single_pattern_Persistent Suicidal 

🧠 Interpretations using 'cognitive_pattern' prompt:
Interpreting 'I've noticed that my mind tends to wander back to the possibility of death, but I'm taking it as a sign that I'm still grappling with some deep-seated concerns and fears. Rather than getting stuck on them, I'm using these thoughts as an opportunity to reflect on what's driving this preoccupation and how I can work through it in a healthy way – whether that means seeking support from loved ones or engaging in self-care practices like journaling or meditation. It's not always easy, but acknowledging my emotions and being kind to myself is helping me build resilience and better coping mechanisms.' with 'This neural pattern represents the cognitive process of _ '


100%|██████████| 1/1 [00:02<00:00,  2.35s/it]


...

Layer 25, Token  2 ('''):
  🤖 24 hour time in English_.

Neural patterns are complex and multifaceted, and this particular pattern

Interpreting 'I've noticed that my mind tends to wander back to the possibility of death, but I'm taking it as a sign that I'm still grappling with some deep-seated concerns and fears. Rather than getting stuck on them, I'm using these thoughts as an opportunity to reflect on what's driving this preoccupation and how I can work through it in a healthy way – whether that means seeking support from loved ones or engaging in self-care practices like journaling or meditation. It's not always easy, but acknowledging my emotions and being kind to myself is helping me build resilience and better coping mechanisms.' with 'This neural pattern represents the cognitive process of _ '


100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


Layer 25, Token  4 ('noticed'):
  🤖 a growing number of neuroscientists believe that the basic building blocks of human experience are formed through pattern recognition

Interpreting 'I've noticed that my mind tends to wander back to the possibility of death, but I'm taking it as a sign that I'm still grappling with some deep-seated concerns and fears. Rather than getting stuck on them, I'm using these thoughts as an opportunity to reflect on what's driving this preoccupation and how I can work through it in a healthy way – whether that means seeking support from loved ones or engaging in self-care practices like journaling or meditation. It's not always easy, but acknowledging my emotions and being kind to myself is helping me build resilience and better coping mechanisms.' with 'This neural pattern represents the cognitive process of _ '


100%|██████████| 1/1 [00:02<00:00,  2.14s/it]

Layer 25, Token  6 ('my'):
  🤖 computer or software application that takes input and produces output based on a predefined set of rules or learned patterns

✅ Single pattern analysis complete!


## ⚗️ Experiment 2: Cognitive Pattern Vector Arithmetic

Perform vector arithmetic between different cognitive patterns to explore therapeutic transformations.

In [ ]:
# ===== EXPERIMENT PARAMETERS =====

# Pattern selection for arithmetic experiments
BASE_PATTERN_TYPE = "self-evaluative"  # Base pattern type
POSITIVE_PATTERN_TYPE = "compassion"  # Pattern to add
SUBTRACT_PATTERN_TYPE = "anxiety"  # Pattern to subtract

# Vector arithmetic parameters
EXTRACTION_LAYER = 15  # Layer to extract activations from
INJECTION_LAYER = 10   # Layer to inject result into
TARGET_TOKEN_POSITION = 3  # Which token position to use (adjust after tokenization)

# Arithmetic weights
POSITIVE_WEIGHT = 1.0
NEGATIVE_WEIGHT = 0.8

# Interpretation settings
ARITHMETIC_INTERPRETATION_PROMPT = "therapeutic_concept"

# ===== EXPERIMENT EXECUTION =====

print("⚗️ EXPERIMENT 2: Cognitive Pattern Vector Arithmetic")
print("=" * 60)

# Get patterns for arithmetic
base_pattern = lab.get_random_pattern(pattern_type=BASE_PATTERN_TYPE)
positive_pattern = lab.get_random_pattern(pattern_type=POSITIVE_PATTERN_TYPE)
subtract_pattern = lab.get_random_pattern(pattern_type=SUBTRACT_PATTERN_TYPE)

patterns_found = [p for p in [base_pattern, positive_pattern, subtract_pattern] if p]

if len(patterns_found) < 2:
    print("❌ Need at least 2 patterns for arithmetic. Using available patterns instead.")
    # Use first few patterns as fallback
    base_pattern = positive_patterns[0] if len(positive_patterns) > 0 else None
    positive_pattern = positive_patterns[1] if len(positive_patterns) > 1 else None
    subtract_pattern = positive_patterns[2] if len(positive_patterns) > 2 else None
    patterns_found = [p for p in [base_pattern, positive_pattern, subtract_pattern] if p]

if len(patterns_found) >= 2:
    print(f"🧮 Arithmetic Experiment: Base + Positive - Negative")
    
    if base_pattern:
        base_text = base_pattern['positive_thought_pattern']
        print(f"\n🔸 Base Pattern ({base_pattern['cognitive_pattern_name']}):")
        print(f"  '{base_text[:80]}...'")
        
        # Get activations for base
        base_tokens, base_token_strings = lab.tokenize_and_display(base_text, 10)
        base_activations = lab.extract_activations(
            text=base_text,
            layers=[EXTRACTION_LAYER],
            token_positions=[min(TARGET_TOKEN_POSITION, len(base_tokens)-1)],
            cache_key="arithmetic_base"
        )
    
    operations = []
    
    if positive_pattern:
        pos_text = positive_pattern['positive_thought_pattern']
        print(f"\n➕ Positive Pattern ({positive_pattern['cognitive_pattern_name']}):")
        print(f"  '{pos_text[:80]}...'")
        
        pos_tokens, _ = lab.tokenize_and_display(pos_text, 5)
        pos_activations = lab.extract_activations(
            text=pos_text,
            layers=[EXTRACTION_LAYER],
            token_positions=[min(TARGET_TOKEN_POSITION, len(pos_tokens)-1)],
            cache_key="arithmetic_positive"
        )
        
        operations.append(("+", pos_activations, EXTRACTION_LAYER, 
                         min(TARGET_TOKEN_POSITION, len(pos_tokens)-1), POSITIVE_WEIGHT))
    
    if subtract_pattern:
        neg_text = subtract_pattern['positive_thought_pattern']
        print(f"\n➖ Subtract Pattern ({subtract_pattern['cognitive_pattern_name']}):")
        print(f"  '{neg_text[:80]}...'")
        
        neg_tokens, _ = lab.tokenize_and_display(neg_text, 5)
        neg_activations = lab.extract_activations(
            text=neg_text,
            layers=[EXTRACTION_LAYER],
            token_positions=[min(TARGET_TOKEN_POSITION, len(neg_tokens)-1)],
            cache_key="arithmetic_negative"
        )
        
        operations.append(("-", neg_activations, EXTRACTION_LAYER,
                         min(TARGET_TOKEN_POSITION, len(neg_tokens)-1), NEGATIVE_WEIGHT))
    
    if base_pattern and operations:
        # Perform vector arithmetic
        print(f"\n🔢 Performing arithmetic at layer {EXTRACTION_LAYER}:")
        result_vector = lab.vector_arithmetic(
            base_activations=base_activations,
            base_layer=EXTRACTION_LAYER,
            base_token=min(TARGET_TOKEN_POSITION, len(base_tokens)-1),
            operations=operations
        )
        
        # Interpret the result
        print(f"\n🧠 Interpreting result vector (injection at layer {INJECTION_LAYER}):")
        interpretation = lab.interpret_vector(
            vector=result_vector,
            injection_layer=INJECTION_LAYER,
            prompt_name=ARITHMETIC_INTERPRETATION_PROMPT,
            max_tokens=25
        )
        
        print(f"🤖 Result: {interpretation}")
        
        # Try different interpretation prompts
        print(f"\n🎭 Alternative Interpretations:")
        for prompt_name in ["emotional_state", "resilience", "coping_mechanism"]:
            alt_interpretation = lab.interpret_vector(
                vector=result_vector,
                injection_layer=INJECTION_LAYER,
                prompt_name=prompt_name,
                max_tokens=20
            )
            print(f"  {prompt_name.replace('_', ' ').title()}: {alt_interpretation}")
    
    print("\n✅ Vector arithmetic experiment complete!")

## 🎨 Experiment 3: Multi-Layer Activation Blending

Extract activations from multiple layers and blend them for complex therapeutic representations.

In [ ]:
# ===== EXPERIMENT PARAMETERS =====

# Pattern and layer selection
BLENDING_PATTERN_TYPE = "resilience"  # Type of pattern to analyze
SOURCE_LAYERS = [5, 10, 15, 20, 25]  # Layers to extract from
LAYER_WEIGHTS = [0.1, 0.2, 0.3, 0.3, 0.1]  # Weights for blending (must sum to 1.0)
TARGET_TOKENS = [2, 4, 6]  # Token positions to analyze

# Injection parameters
INJECTION_LAYERS = [8, 12, 18]  # Where to inject blended vectors
BLENDING_PROMPTS = ["resilience", "growth_mindset", "self_compassion"]  # Interpretation contexts

# ===== EXPERIMENT EXECUTION =====

print("🎨 EXPERIMENT 3: Multi-Layer Activation Blending")
print("=" * 60)

# Validate parameters
if abs(sum(LAYER_WEIGHTS) - 1.0) > 0.01:
    print(f"⚠️ Layer weights sum to {sum(LAYER_WEIGHTS):.3f}, normalizing...")
    total = sum(LAYER_WEIGHTS)
    LAYER_WEIGHTS = [w/total for w in LAYER_WEIGHTS]

if len(SOURCE_LAYERS) != len(LAYER_WEIGHTS):
    print(f"❌ Mismatch: {len(SOURCE_LAYERS)} layers but {len(LAYER_WEIGHTS)} weights")
else:
    # Get pattern - try resilience first, then any pattern as fallback
    pattern = lab.get_random_pattern(pattern_type=BLENDING_PATTERN_TYPE)
    
    if not pattern:
        print(f"❌ No pattern found for type: {BLENDING_PATTERN_TYPE}, using first available pattern")
        pattern = positive_patterns[0] if positive_patterns else None
    
    if not pattern:
        print(f"❌ No patterns available")
    else:
        text = pattern['positive_thought_pattern']
        print(f"🎯 Pattern: {pattern['cognitive_pattern_name']}")
        print(f"📝 Text: '{text[:100]}...'")
        
        # Tokenize
        tokens, token_strings = lab.tokenize_and_display(text, 12)
        
        # Validate token positions
        valid_tokens = [t for t in TARGET_TOKENS if 0 <= t < len(tokens)]
        if len(valid_tokens) != len(TARGET_TOKENS):
            print(f"⚠️ Some token positions invalid, using: {valid_tokens}")
            TARGET_TOKENS = valid_tokens
        
        print(f"\n🎯 Target tokens:")
        for pos in TARGET_TOKENS:
            print(f"  {pos:2d}: '{token_strings[pos].strip()}'")
        
        # Extract activations from all source layers
        activations = lab.extract_activations(
            text=text,
            layers=SOURCE_LAYERS,
            token_positions=TARGET_TOKENS,
            cache_key=f"blending_{pattern['cognitive_pattern_name'][:15]}"
        )
        
        # Blend activations for each token
        print(f"\n🎨 Blending activations with weights: {[f'{w:.2f}' for w in LAYER_WEIGHTS]}")
        
        blended_vectors = {}
        
        for token_idx, token_pos in enumerate(TARGET_TOKENS):
            # Initialize with zeros
            if isinstance(activations[SOURCE_LAYERS[0]], list):
                blended = torch.zeros_like(activations[SOURCE_LAYERS[0]][token_idx])
            else:
                blended = torch.zeros_like(activations[SOURCE_LAYERS[0]][:, token_pos, :])
            
            # Weighted sum across layers
            for layer_idx, (layer, weight) in enumerate(zip(SOURCE_LAYERS, LAYER_WEIGHTS)):
                if isinstance(activations[layer], list):
                    activation = activations[layer][token_idx]
                else:
                    activation = activations[layer][:, token_pos, :]
                
                blended += weight * activation
            
            blended_vectors[token_pos] = blended
            print(f"  ✅ Blended vector for token {token_pos} ('{token_strings[token_pos].strip()}')")
        
        # Interpret blended vectors across different injection layers and prompts
        print(f"\n🧠 Interpretations across {len(INJECTION_LAYERS)} injection layers:")
        print("=" * 80)
        
        for inject_layer in INJECTION_LAYERS:
            print(f"\n📍 Injection Layer {inject_layer}:")
            
            for token_pos in TARGET_TOKENS[:2]:  # Focus on first 2 tokens
                token_text = token_strings[token_pos].strip()
                print(f"\n  🔸 Token '{token_text}' interpretations:")
                
                for prompt_name in BLENDING_PROMPTS:
                    interpretation = lab.interpret_vector(
                        vector=blended_vectors[token_pos],
                        injection_layer=inject_layer,
                        prompt_name=prompt_name,
                        max_tokens=18
                    )
                    
                    print(f"    {prompt_name.replace('_', ' ').title():.<18} {interpretation}")
        
        # Analyze blending statistics
        print(f"\n📊 Blending Statistics:")
        for token_pos in TARGET_TOKENS:
            vector = blended_vectors[token_pos]
            flat = vector.flatten()
            
            print(f"  Token {token_pos:2d} ('{token_strings[token_pos].strip()}'):")
            print(f"    Norm: {torch.norm(flat):.1f}, Mean: {flat.mean():.3f}, Std: {flat.std():.3f}")
        
        print("\n✅ Multi-layer blending experiment complete!")

## 🔄 Experiment 4: Negative-to-Positive Pattern Transformation

Use vector arithmetic to transform negative thought patterns into positive ones using the dataset's reference examples.

In [ ]:
# ===== EXPERIMENT PARAMETERS =====

# Transformation parameters
TRANSFORMATION_LAYER = 16  # Layer for extracting transformation vectors
INJECTION_LAYER = 12       # Layer for interpretation
TOKEN_POSITION = 5         # Focus token (adjust after seeing tokenization)

# Pattern selection - use patterns with both positive and negative examples
FOCUS_PATTERN_TYPES = ["self-evaluative", "anxiety", "depression"]  # Types with good neg/pos examples
NUM_TRANSFORMATIONS = 3    # How many transformations to try

# Interpretation prompts for transformation analysis
TRANSFORMATION_PROMPTS = ["therapeutic_concept", "resilience", "mindfulness"]

# ===== EXPERIMENT EXECUTION =====

print("🔄 EXPERIMENT 4: Negative-to-Positive Pattern Transformation")
print("=" * 60)

transformation_results = []

for transform_idx in range(NUM_TRANSFORMATIONS):
    print(f"\n🎯 Transformation {transform_idx + 1}/{NUM_TRANSFORMATIONS}")
    print("=" * 40)
    
    # Find pattern with both negative and positive examples
    suitable_patterns = []
    for pattern in positive_patterns:
        has_negative = 'reference_negative_example' in pattern and pattern['reference_negative_example']
        has_positive = pattern['positive_thought_pattern']
        # Use all patterns since they all have both positive and negative examples
        
        if has_negative and has_positive:
            suitable_patterns.append(pattern)
    
    if not suitable_patterns:
        print("❌ No suitable patterns found with both positive and negative examples")
        continue
    
    # Select pattern (rotate through available patterns)
    pattern = suitable_patterns[transform_idx % len(suitable_patterns)]
    positive_text = pattern['positive_thought_pattern']
    negative_text = pattern['reference_negative_example']
    
    print(f"📋 Pattern: {pattern['cognitive_pattern_name']}")
    print(f"📝 Type: {pattern['cognitive_pattern_type']}")
    
    print(f"\n❌ Negative: '{negative_text[:80]}...'")
    print(f"✅ Positive: '{positive_text[:80]}...'")
    
    # Tokenize both versions
    print(f"\n🔤 Tokenizing negative version:")
    neg_tokens, neg_token_strings = lab.tokenize_and_display(negative_text, 10)
    
    print(f"\n🔤 Tokenizing positive version:")
    pos_tokens, pos_token_strings = lab.tokenize_and_display(positive_text, 10)
    
    # Adjust token position if needed
    actual_token_pos = min(TOKEN_POSITION, min(len(neg_tokens), len(pos_tokens)) - 1)
    
    print(f"\n🎯 Using token position {actual_token_pos}:")
    if actual_token_pos < len(neg_token_strings):
        print(f"  Negative: '{neg_token_strings[actual_token_pos].strip()}'")
    if actual_token_pos < len(pos_token_strings):
        print(f"  Positive: '{pos_token_strings[actual_token_pos].strip()}'")
    
    # Extract activations
    neg_activations = lab.extract_activations(
        text=negative_text,
        layers=[TRANSFORMATION_LAYER],
        token_positions=[actual_token_pos],
        cache_key=f"neg_{transform_idx}_{pattern['cognitive_pattern_name'][:10]}"
    )
    
    pos_activations = lab.extract_activations(
        text=positive_text,
        layers=[TRANSFORMATION_LAYER],
        token_positions=[actual_token_pos],
        cache_key=f"pos_{transform_idx}_{pattern['cognitive_pattern_name'][:10]}"
    )
    
    # Compute transformation vector: positive - negative
    print(f"\n🧮 Computing transformation vector (positive - negative):")
    
    if isinstance(pos_activations[TRANSFORMATION_LAYER], list):
        pos_vector = pos_activations[TRANSFORMATION_LAYER][0]
        neg_vector = neg_activations[TRANSFORMATION_LAYER][0]
    else:
        pos_vector = pos_activations[TRANSFORMATION_LAYER][:, actual_token_pos, :]
        neg_vector = neg_activations[TRANSFORMATION_LAYER][:, actual_token_pos, :]
    
    transformation_vector = pos_vector - neg_vector
    
    # Analyze transformation vector
    flat_transform = transformation_vector.flatten()
    print(f"  Transformation vector stats:")
    print(f"    Norm: {torch.norm(flat_transform):.1f}")
    print(f"    Mean: {flat_transform.mean():.3f}")
    print(f"    Std: {flat_transform.std():.3f}")
    
    # Interpret the transformation vector
    print(f"\n🧠 Transformation Vector Interpretations:")
    
    transform_interpretations = {}
    
    for prompt_name in TRANSFORMATION_PROMPTS:
        interpretation = lab.interpret_vector(
            vector=transformation_vector,
            injection_layer=INJECTION_LAYER,
            prompt_name=prompt_name,
            max_tokens=20
        )
        
        transform_interpretations[prompt_name] = interpretation
        print(f"  {prompt_name.replace('_', ' ').title():.<20} {interpretation}")
    
    # Test transformation: apply to negative to get positive-like
    print(f"\n🔄 Testing transformation (negative + transform_vector):")
    
    transformed_vector = neg_vector + transformation_vector
    
    for prompt_name in ["cognitive_pattern", "emotional_state"]:
        result_interpretation = lab.interpret_vector(
            vector=transformed_vector,
            injection_layer=INJECTION_LAYER,
            prompt_name=prompt_name,
            max_tokens=25
        )
        
        print(f"  {prompt_name.replace('_', ' ').title()}: {result_interpretation}")
    
    # Store results
    transformation_results.append({
        'pattern_name': pattern['cognitive_pattern_name'],
        'pattern_type': pattern['cognitive_pattern_type'],
        'negative_text': negative_text[:100],
        'positive_text': positive_text[:100],
        'token_position': actual_token_pos,
        'transformation_norm': float(torch.norm(flat_transform)),
        'interpretations': transform_interpretations
    })

print(f"\n📊 Transformation Summary:")
print("=" * 50)

if transformation_results:
    avg_norm = np.mean([r['transformation_norm'] for r in transformation_results])
    print(f"Average transformation vector norm: {avg_norm:.1f}")
    
    print(f"\nPattern types analyzed:")
    for result in transformation_results:
        print(f"  - {result['pattern_name']} ({result['pattern_type']})")

print("\n✅ Negative-to-positive transformation experiment complete!")

## 🔬 Custom Experiment Sandbox

Free-form experimentation space. Modify the parameters and code below to run your own custom experiments.

In [ ]:
# ===== CUSTOM EXPERIMENT SANDBOX =====
# Feel free to modify anything below for your own experiments!

print("🔬 CUSTOM EXPERIMENT SANDBOX")
print("=" * 50)

# Example: Compare specific patterns from our hardcoded data
PATTERN_A_NAME = "Self-Critical"  # Adjust based on available patterns
PATTERN_B_NAME = "Anxiety"  # Adjust based on available patterns

# Search for patterns by partial name match
pattern_a = None
pattern_b = None

# Find patterns by name
for pattern in positive_patterns:
    if PATTERN_A_NAME.lower() in pattern['cognitive_pattern_name'].lower():
        pattern_a = pattern
        break

for pattern in positive_patterns:
    if PATTERN_B_NAME.lower() in pattern['cognitive_pattern_name'].lower():
        pattern_b = pattern
        break

if pattern_a and pattern_b:
    print(f"🔍 Comparing:")
    print(f"  A: {pattern_a['cognitive_pattern_name']}")
    print(f"  B: {pattern_b['cognitive_pattern_name']}")
    
    # Extract and compare activations
    text_a = pattern_a['positive_thought_pattern']
    text_b = pattern_b['positive_thought_pattern']
    
    print(f"\n💭 Pattern A: '{text_a[:60]}...'")
    print(f"💭 Pattern B: '{text_b[:60]}...'")
    
    # Your analysis code here...
    print("\n💡 Add your custom analysis here!")
    print("Examples:")
    print("- Extract activations from both patterns")
    print("- Compute cosine similarity between vectors")
    print("- Perform pattern_a - pattern_b arithmetic")
    print("- Visualize activation differences")
    print("- Try different interpretation prompts")
    
    # Example custom analysis:
    print("\n🧮 Sample Analysis:")
    print("# Extract activations from layer 15, token 3")
    print("# activations_a = lab.extract_activations(text_a, layers=[15], token_positions=[3])")
    print("# activations_b = lab.extract_activations(text_b, layers=[15], token_positions=[3])")
    print("# difference = activations_a[15][0] - activations_b[15][0]")
    print("# interpretation = lab.interpret_vector(difference, 12, 'therapeutic_concept')")

elif not pattern_a and not pattern_b:
    print(f"❌ Neither pattern found. Available pattern names:")
    unique_names = set(p['cognitive_pattern_name'] for p in positive_patterns)
    for name in sorted(unique_names):
        print(f"  - {name}")
        
    print(f"\n🔍 Quick examples you can try:")
    print(f"  - Change PATTERN_A_NAME to 'Rumination'")
    print(f"  - Change PATTERN_B_NAME to 'Hope'") 
    print(f"  - Change PATTERN_A_NAME to 'Social'")
    
else:
    found_pattern = pattern_a or pattern_b
    missing_name = PATTERN_B_NAME if pattern_a else PATTERN_A_NAME
    print(f"✅ Found: {found_pattern['cognitive_pattern_name']}")
    print(f"❌ Missing: Pattern with '{missing_name}' in name")
    
    print(f"\nAvailable pattern names:")
    unique_names = set(p['cognitive_pattern_name'] for p in positive_patterns)
    for name in sorted(unique_names):
        print(f"  - {name}")

print(f"\n🎉 Sandbox ready for your experiments!")
print(f"\n📋 Quick Tips:")
print(f"  - All patterns have both positive and negative examples")
print(f"  - Use lab.get_random_pattern() to get a random pattern")
print(f"  - Use lab.get_random_pattern(pattern_type='anxiety') for specific types")
print(f"  - Pattern types: {list(set(p['cognitive_pattern_type'] for p in positive_patterns))}")

## 📊 Lab Results Visualization

Visualize and analyze results from your experiments.

In [ ]:
# Visualization and analysis of lab results
print("📊 LAB RESULTS VISUALIZATION")
print("=" * 50)

# Check if we have transformation results to visualize
if 'transformation_results' in locals() and transformation_results:
    print(f"\n📈 Transformation Analysis (n={len(transformation_results)}):")
    
    # Plot transformation vector norms
    norms = [r['transformation_norm'] for r in transformation_results]
    pattern_names = [r['pattern_name'][:20] for r in transformation_results]
    
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(norms)), norms)
    plt.xlabel('Transformation Experiment')
    plt.ylabel('Vector Norm')
    plt.title('Transformation Vector Magnitudes')
    plt.xticks(range(len(pattern_names)), pattern_names, rotation=45)
    plt.tight_layout()
    plt.show()
    
    print(f"Average transformation norm: {np.mean(norms):.2f} ± {np.std(norms):.2f}")

# Model and dataset statistics
if 'MODEL_INFO' in locals():
    print(f"\n🏗️ Model Information:")
    for key, value in MODEL_INFO.items():
        print(f"  {key.replace('_', ' ').title()}: {value}")

print(f"\n📂 Dataset Statistics:")
print(f"  Total patterns: {len(positive_patterns)}")
print(f"  Cached activations: {len(lab.activation_cache)}")

# Cache information
if lab.activation_cache:
    print(f"\n💾 Activation Cache:")
    for cache_key in lab.activation_cache.keys():
        print(f"  - {cache_key}")

print("\n✅ Lab session complete! Results visualized.")
print("\n🔬 To run more experiments:")
print("  1. Modify parameters in any experiment cell above")
print("  2. Re-run the experiment cell")
print("  3. Cached activations will speed up repeated runs")
print("  4. Use the sandbox for custom experiments")

## 🧠 Lab Summary and Next Steps

Summary of the Psychiatry Vector Arithmetic Laboratory and suggestions for further exploration.

In [ ]:
print("🧠 PSYCHIATRY VECTOR ARITHMETIC LAB SUMMARY")
print("=" * 60)

print(f"\n🔬 Lab Components:")
print(f"  ✅ Model loaded: {MODEL_INFO.get('name', 'Unknown')} on {MODEL_INFO.get('device', 'Unknown')}")
print(f"  ✅ Hardcoded patterns: {len(positive_patterns)} psychiatry-focused thought patterns")
print(f"  ✅ Therapeutic prompts: {len(THERAPEUTIC_PROMPTS)} specialized interpretations")
print(f"  ✅ Lab utilities: Modular experimentation framework")

print(f"\n🧪 Experiments Available:")
print(f"  1️⃣  Single Pattern Analysis - Layer-wise interpretation of individual patterns")
print(f"  2️⃣  Vector Arithmetic - Combine/subtract cognitive patterns")
print(f"  3️⃣  Multi-Layer Blending - Weight and combine activations from multiple layers")
print(f"  4️⃣  Negative→Positive Transformation - Learn therapeutic transformation vectors")
print(f"  🔬 Custom Sandbox - Free-form experimentation space")

print(f"\n🎭 Therapeutic Interpretation Contexts:")
for name in THERAPEUTIC_PROMPTS.keys():
    print(f"  - {name.replace('_', ' ').title()}")

print(f"\n📊 Hardcoded Cognitive Patterns:")
if positive_patterns:
    pattern_types = list(set(p['cognitive_pattern_type'] for p in positive_patterns))
    for ptype in sorted(pattern_types):
        count = sum(1 for p in positive_patterns if p['cognitive_pattern_type'] == ptype)
        print(f"  - {ptype} ({count} example)")

print(f"\n💡 Psychiatry-Focused Patterns Include:")
print(f"  🧠 Self-critical rumination and compassionate reframing")
print(f"  😰 Anxiety catastrophizing and grounded reality checks")  
print(f"  😔 Depression hopelessness and hope cultivation")
print(f"  🎯 Perfectionism paralysis and progress-focused mindset")
print(f"  👥 Social anxiety and confidence building")
print(f"  🛡️ Trauma responses and safety/grounding techniques")
print(f"  🌊 Emotional overwhelm and regulation strategies")
print(f"  🧘 Mindfulness and present-moment awareness")
print(f"  💝 Self-compassion and inner kindness")
print(f"  💪 Resilience recognition and strength building")

print(f"\n🚀 Next Steps and Ideas:")
print(f"  🔄 Run experiments with different model layers (early vs late)")
print(f"  🎯 Focus on specific therapeutic domains (anxiety, depression, resilience)")
print(f"  📏 Compare vector arithmetic results across different models")
print(f"  🔍 Analyze which layers best capture therapeutic concepts")
print(f"  🎨 Create therapeutic 'concept directions' for model steering")
print(f"  📊 Build visualizations of cognitive pattern spaces")
print(f"  🤝 Combine multiple positive patterns for enhanced interventions")

print(f"\n💡 Advanced Research Directions:")
print(f"  - Investigate layer-specific therapeutic representations")
print(f"  - Build 'cognitive transformation functions' using vector arithmetic")
print(f"  - Develop automated therapeutic prompt generation")
print(f"  - Create personalized therapeutic vector profiles")
print(f"  - Study cross-modal therapeutic representations")

print(f"\n🎉 Happy experimenting! The lab is ready for therapeutic AI research.")
print(f"✨ No external datasets required - all patterns are self-contained!")

# Quick reference
print(f"\n📋 Quick Reference:")
print(f"  lab.get_random_pattern()  # Get random pattern")
print(f"  lab.get_random_pattern(pattern_type='anxiety')  # Get specific type")
print(f"  lab.tokenize_and_display(text)")
print(f"  lab.extract_activations(text, layers=[10, 15], token_positions=[2, 4])")
print(f"  lab.interpret_activation(text, layer=15, token_position=3, prompt_name='resilience')")
print(f"  lab.vector_arithmetic(base_acts, layer, token, operations)")
print(f"  lab.interpret_vector(vector, injection_layer=12, prompt_name='mindfulness')")